In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

In [2]:
# GRADED FUNCTION: zero_pad

def zero_pad(X, pad):
    X_pad = np.pad(X, ((0,0), (pad,pad), (pad,pad), (0,0)), 'constant')
    return X_pad

In [3]:
#一个卷积

def conv_single_step(a_slice_prev, W, b):

    s = a_slice_prev * W
    Z = np.sum(s)
    Z = Z + b
    return Z
#测试
np.random.seed(1)
a_slice_prev = np.random.randn(4, 4, 3)
W = np.random.randn(4, 4, 3)
b = np.random.randn(1, 1, 1)
print(a_slice_prev)
print(W)
print(b)

Z = conv_single_step(a_slice_prev, W, b)
print("Z =", Z)

[[[ 1.62434536 -0.61175641 -0.52817175]
  [-1.07296862  0.86540763 -2.3015387 ]
  [ 1.74481176 -0.7612069   0.3190391 ]
  [-0.24937038  1.46210794 -2.06014071]]

 [[-0.3224172  -0.38405435  1.13376944]
  [-1.09989127 -0.17242821 -0.87785842]
  [ 0.04221375  0.58281521 -1.10061918]
  [ 1.14472371  0.90159072  0.50249434]]

 [[ 0.90085595 -0.68372786 -0.12289023]
  [-0.93576943 -0.26788808  0.53035547]
  [-0.69166075 -0.39675353 -0.6871727 ]
  [-0.84520564 -0.67124613 -0.0126646 ]]

 [[-1.11731035  0.2344157   1.65980218]
  [ 0.74204416 -0.19183555 -0.88762896]
  [-0.74715829  1.6924546   0.05080775]
  [-0.63699565  0.19091548  2.10025514]]]
[[[ 0.12015895  0.61720311  0.30017032]
  [-0.35224985 -1.1425182  -0.34934272]
  [-0.20889423  0.58662319  0.83898341]
  [ 0.93110208  0.28558733  0.88514116]]

 [[-0.75439794  1.25286816  0.51292982]
  [-0.29809284  0.48851815 -0.07557171]
  [ 1.13162939  1.51981682  2.18557541]
  [-1.39649634 -1.44411381 -0.50446586]]

 [[ 0.16003707  0.87616892  

In [4]:
#卷积神经网络

#卷积前向传播
def conv_forward(A_prev, W, b, hparameters):


    # 从A_prev中检索尺寸 
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape

    # 从W中检索尺寸
    (f, f, n_C_prev, n_C) = W.shape

    # 从hparameters中检索尺寸
    stride = hparameters["stride"]
    pad = hparameters["pad"]

    #计算 CONV 输出体积的尺寸
    n_H = int((n_H_prev - f + 2*pad) / stride + 1)
    n_W = int((n_W_prev - f + 2*pad) / stride + 1)

    #用零初始化输出体积Z
    Z = np.zeros((m, n_H, n_W, n_C))


    A_prev_pad = zero_pad(A_prev, pad)

    for i in range(m):                                
        a_prev_pad = A_prev_pad[i, :, :, :]            
        for h in range(n_H):                           
            for w in range(n_W):                       
                for c in range(n_C):                   
                    vert_start = stride * h
                    vert_end = vert_start + f
                    horiz_start = stride * w
                    horiz_end = horiz_start + f

                    # 使用角定义a_prev_pad的3D切片
                    a_slice_prev = a_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :]

                    # 使用过滤器和偏置进行卷积切片，返回一个输出神经元
                    Z[i, h, w, c] = conv_single_step(a_slice_prev, W[:, :, :, c], b[:, :, :, c])

    ### END CODE HERE ###

    #验证
    assert(Z.shape == (m, n_H, n_W, n_C))

    # 信息保存在反向传播的缓存中
    cache = (A_prev, W, b, hparameters)

    return Z, cache
np.random.seed(1)
A_prev = np.random.randn(10,4,4,3)
W = np.random.randn(2,2,3,8)
b = np.random.randn(1,1,1,8)
hparameters = {"pad" : 2,
               "stride": 2}

Z, cache_conv = conv_forward(A_prev, W, b, hparameters)
print("Z's mean =", np.mean(Z))
print("Z[3,2,1] =", Z[3,2,1])
print("cache_conv[0][1][2][3] =", cache_conv[0][1][2][3])

Z's mean = 0.048995203528855794
Z[3,2,1] = [-0.61490741 -6.7439236  -2.55153897  1.75698377  3.56208902  0.53036437
  5.18531798  8.75898442]
cache_conv[0][1][2][3] = [-0.20075807  0.18656139  0.41005165]


In [5]:
#池化层前向传播
def pool_forward(A_prev, hparameters, mode = "max"):

    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape

    #超参数
    f = hparameters["f"]
    stride = hparameters["stride"]

    # 定义输出维度
    n_H = int(1 + (n_H_prev - f) / stride)
    n_W = int(1 + (n_W_prev - f) / stride)
    n_C = n_C_prev

    #初始化输出矩阵A
    A = np.zeros((m, n_H, n_W, n_C))              


    for i in range(m):                   
        for h in range(n_H):                     # 垂直轴
            for w in range(n_W):                 # 水平轴
                for c in range (n_C):           
                    vert_start = h * stride
                    vert_end = vert_start + f
                    horiz_start = w * stride
                    horiz_end = horiz_start + f

                    a_prev_slice = A_prev[i, vert_start:vert_end, horiz_start:horiz_end, c]
                    # 计算切片上的池化操作
                    if mode == "max":
                        A[i, h, w, c] = np.max(a_prev_slice)
                    elif mode == "average":
                        A[i, h, w, c] = np.mean(a_prev_slice)



    # 将输入和hparameters存储在pool_backward()的缓存中
    cache = (A_prev, hparameters)

    #验证
    assert(A.shape == (m, n_H, n_W, n_C))

    return A, cache
np.random.seed(1)
A_prev = np.random.randn(2, 4, 4, 3)
hparameters = {"stride" : 2, "f": 3}

A, cache = pool_forward(A_prev, hparameters)
print("mode = max")
print("A =", A)
print()
A, cache = pool_forward(A_prev, hparameters, mode = "average")
print("mode = average")
print("A =", A)

mode = max
A = [[[[1.74481176 0.86540763 1.13376944]]]


 [[[1.13162939 1.51981682 2.18557541]]]]

mode = average
A = [[[[ 0.02105773 -0.20328806 -0.40389855]]]


 [[[-0.22154621  0.51716526  0.48155844]]]]


In [6]:
#卷积层后向传播
def conv_backward(dZ, cache):

    # 从缓存中检索信息
    (A_prev, W, b, hparameters) = cache

    #从A_prev中检索尺寸
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape

    #从 W 中检索尺寸
    (f, f, n_C_prev, n_C) = W.shape

    #从“hparameters”中检索信息
    stride = hparameters['stride']
    pad = hparameters['pad']

    # 从 dZ 中检索尺寸
    (m, n_H, n_W, n_C) = dZ.shape

    #初始化dA_prev、dW、db
    dA_prev = np.zeros((m, n_H_prev, n_W_prev, n_C_prev))
    dW = np.zeros((f, f, n_C_prev, n_C))
    db = np.zeros((1, 1, 1, n_C))

    #填充A_prev_pad，dA_prev_pad
    A_prev_pad = zero_pad(A_prev, pad)
    dA_prev_pad = zero_pad(dA_prev, pad)

    for i in range(m):                       # loop over the training examples

        # 从A_prev_pad和dA_prev_pad中选择第 i 个训练示例
        a_prev_pad = A_prev_pad[i, :, :, :]
        da_prev_pad = dA_prev_pad[i, :, :, :]

        for h in range(n_H):                   
            for w in range(n_W):              
                for c in range(n_C):          

                    #当前切片的corners
                    vert_start = h * stride
                    vert_end = vert_start + f
                    horiz_start = w * stride
                    horiz_end = horiz_start + f

                    a_slice = a_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :]

                    # 更新dw和db
                    da_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :] += W[:,:,:,c] * dZ[i, h, w, c]
                    dW[:,:,:,c] += a_slice * dZ[i, h, w, c]
                    db[:,:,:,c] += dZ[i, h, w, c]

        # Set the ith training example's dA_prev to the unpaded da_prev_pad (Hint: use X[pad:-pad, pad:-pad, :])
        dA_prev[i, :, :, :] = da_prev_pad[pad:-pad, pad:-pad, :]

    # 验证
    assert(dA_prev.shape == (m, n_H_prev, n_W_prev, n_C_prev))

    return dA_prev, dW, db

In [7]:
#最大池后向传播
def create_mask_from_window(x):

    mask = (x == np.max(x))

    return mask

In [8]:
#平均池后向传播
def distribute_value(dz, shape):
   
    (n_H, n_W) = shape

    average = dz / (n_H * n_W)

    #  创建一个矩阵储存平均值
    a = average * np.ones(shape)
 
    return a

In [9]:
#池化层后向传播
def pool_backward(dA, cache, mode = "max"):

    # 从缓存中检索信息
    (A_prev, hparameters) = cache

    #从hparameters检索超参数
    stride = hparameters['stride']
    f = hparameters['f']

    #从A_prev和 dA 的大小中检索尺寸（≈2 行）
    m, n_H_prev, n_W_prev, n_C_prev = A_prev.shape
    m, n_H, n_W, n_C = dA.shape

    # 初始化dA_prev
    dA_prev = np.zeros(np.shape(A_prev))

    for i in range(m):                       

        a_prev = A_prev[i, :, :, :]

        for h in range(n_H):                  
            for w in range(n_W):               
                for c in range(n_C):           

                   
                    vert_start = h * stride
                    vert_end = vert_start + f
                    horiz_start = w * stride
                    horiz_end = horiz_start + f

                    # 计算两种模式下的后向传播。
                    if mode == "max":
                        a_prev_slice = a_prev[vert_start:vert_end, horiz_start:horiz_end, c]
                        mask = create_mask_from_window(a_prev_slice)
                        dA_prev[i, vert_start: vert_end, horiz_start: horiz_end, c] += np.multiply(mask, dA[i, h, w, c])
                    elif mode == "average":
                        da = dA[i, h, w, c]
                        shape = (f, f)
                        dA_prev[i, vert_start: vert_end, horiz_start: horiz_end, c] += distribute_value(da, shape)
    # 验证
    assert(dA_prev.shape == A_prev.shape)

    return dA_prev

np.random.seed(1)
A_prev = np.random.randn(5, 5, 3, 2)
hparameters = {"stride" : 1, "f": 2}
A, cache = pool_forward(A_prev, hparameters)
dA = np.random.randn(5, 4, 2, 2)

dA_prev = pool_backward(dA, cache, mode = "max")
print("mode = max")
print('mean of dA = ', np.mean(dA))
print('dA_prev[1,1] = ', dA_prev[1,1])  
print()
dA_prev = pool_backward(dA, cache, mode = "average")
print("mode = average")
print('mean of dA = ', np.mean(dA))
print('dA_prev[1,1] = ', dA_prev[1,1]) 

mode = max
mean of dA =  0.14571390272918056
dA_prev[1,1] =  [[ 0.          0.        ]
 [ 5.05844394 -1.68282702]
 [ 0.          0.        ]]

mode = average
mean of dA =  0.14571390272918056
dA_prev[1,1] =  [[ 0.08485462  0.2787552 ]
 [ 1.26461098 -0.25749373]
 [ 1.17975636 -0.53624893]]
